In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import plot_importance

In [0]:
#边界，中心
num=(1e-5)
xmin=3750901.5068*num
xmax=3770901.5068*num
ymin=-19268905.6133*num
ymax=-19208905.6133*num
xcenter=(xmin+xmax)/2
ycenter=(ymin+ymax)/2

In [0]:
# 确定过采样的数据条数
dy=data[data['target']==1]
k=len(data)-len(dy)*3

24312

In [0]:
# 读入训练数据
data=pd.read_csv('train_count5.csv')

In [0]:
dy=data[data['target']==1]
#过采样
dy1=data[data['target']==1]
dy2=dy1.sample(n=k,axis=0)
dn=data[data['target']!=1]
# dn=dn.iloc[:dy.shape[0],:]
dyn=pd.concat([dy,dn,dy1,dy2],axis=0)

#训练数据
dyn_data=dyn.drop(['Unnamed: 0','hashid','endx','endy','targetx','targety','target'],axis=1).values

#各种目标
dyn_endx=dyn['endx'].values
dyn_endy=dyn['endy'].values
dyn_targetx=dyn['targetx'].values
dyn_targety=dyn['targety'].values
dyn_target=dyn['target'].values

In [0]:
#xgboost最佳参数
import xgboost as xgb
ps={}
ps['gamma']=[0.05,0.1,0.5]
ps['max_depth']=[10,15,20]
ps['learning_rate']=[0.05,0.1,0.3]
ps['objective']=['binary:logistic']
ps['min_child_weight']=[0.5,1,5]
plst = list(ps.items())
num_rounds = 2000

#筛选最佳超参数
from sklearn.model_selection import GridSearchCV
xgbrg=xgb.XGBClassifier()

m_rx=GridSearchCV(xgbrg,ps)
#由于数据过多，gridsearch吃不消，选取部分数据来检查最佳参数
gindex=np.random.randint(len(dyn),size=4000)
sample=dyn_data[gindex,:]
sample_targetx=dyn_target[gindex]
m_rx.fit(sample,sample_targetx)
print('rx_best_params: ',m_rx.best_params_)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


rx_best_params:  {'gamma': 0.05, 'learning_rate': 0.3, 'max_depth': 10, 'min_child_weight': 0.5, 'objective': 'binary:logistic'}


In [0]:

#训练集，测试集划分
from sklearn.model_selection import train_test_split
train_rx,test_rx,train_targetx,test_targetx=train_test_split(dyn_data,dyn_target,test_size=0.3)

#用xgboost最优参数预测
import xgboost as xgb
xgb_rx=xgb.XGBClassifier(gamma=0.05,learning_rate=0.05,max_depth= 15,
                        min_child_weight= 0.5,n_estimators=300,objective= 'binary:logistic')
xgb_rx.fit(train_rx,train_targetx)
predict_rx=xgb_rx.predict(test_rx)


from sklearn.metrics import precision_score,roc_auc_score

auc_score = roc_auc_score(test_targetx,predict_rx)
pre_score = precision_score(test_targetx,predict_rx)

print("xgb_auc_score:",auc_score)
print("xgb_pre_score:",pre_score)

xgb_auc_score: 0.9704279024925158
xgb_pre_score: 0.9577592275973046


In [0]:
test=pd.read_csv('test_count5.csv')
test.drop(['Unnamed: 0','Unnamed: 0.1','hashid'],inplace=True,axis=1)
test_data=test.values

In [0]:
# 训练数据集，预测测试集
import xgboost as xgb
xgb_rx=xgb.XGBClassifier(gamma=0.05,learning_rate=0.05,max_depth= 15,
                        min_child_weight= 0.5,n_estimators=300,objective= 'binary:logistic')

xgb_rx.fit(dyn_data,dyn_target)

p_rx2=xgb_rx.predict(test_data)


In [0]:
# 生成预测结果
test=pd.read_csv('test_count5.csv')
test['target']=p_rx2

result=test[['hashid','target']]

test_1trace=pd.read_csv('test_1trace.csv')

result1=result.rename(columns={'hashid':'id'})
test_1trace.rename(columns={'hashid':'id'},inplace=True)
test_1trace['target']=0
#test专用数列
l=['id','target']
r=result1[l]
t=test_1trace[l]
result1p=pd.concat([r,t])

result1p.to_csv('t5_os_estim.csv')